In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# ===============================================================
# 1) Excel 데이터 읽어서 dataframe으로 변환
# ===============================================================

# 파일 읽기
df = pd.read_csv("ELS.csv")


In [ ]:
# 필요 함수
# zero_rate을 통해 Discount rate 만들어주는 함수 맞나요..?
def discount_OIS(t, ois_terms=None, ois_rates=None):
    if t <= ois_terms[0]:
        return np.exp(-ois_rates[0] * t)
    if t >= ois_terms[-1]:
        return np.exp(-ois_rates[-1] * t)

    i = np.searchsorted(ois_terms, t) - 1
    df_i = np.exp(-ois_rates[i] * ois_terms[i])
    df_j = np.exp(-ois_rates[i+1] * ois_terms[i+1])
    w = (t - ois_terms[i]) / (ois_terms[i+1] - ois_terms[i])
    lnDF = np.log(df_i) + w * (np.log(df_j) - np.log(df_i))
    return np.exp(lnDF)

In [ ]:
# ===============================================================
# 2) 필요한 데이터 추출
# ===============================================================



# 2021-10-22일 이후 행만 필터링
df['date'] = pd.to_datetime(df['date'])
df_filtered = df[df['date'] >= pd.to_datetime('2021-10-22')].copy()

# 오류를 방지하기 위해 모든 열 제목을 lowercase로 만들기
cols_lower = {c.lower(): c for c in df_filtered.columns}

# 1. Zero rates
horizons = ['3M','6M','1Y','2Y','3Y']
zero_rate_cols = []
# zero_rate에 해당하는 열을 모두 찾아서 저장
for h in horizons:
    found = cols_lower["zero_rate_"+ h.lower()]
    zero_rate_cols.append(found)

# np_array로 만들기
zero_rate = df_filtered[zero_rate_cols].to_numpy()



# Discount OIS 어떻게 쓰누... ㅋㅋ
discount_factor = None


# 2. r - q (r_minus_q)
# 열의 이름은 차후 변경될 수 있음을 유의하자.
rq_k_col = cols_lower['r_q_kospi']
rq_s_col = cols_lower['r_q_samsung']
r_minus_q = df_filtered[[rq_k_col, rq_s_col]].to_numpy()

# 3. vol(sigma)

vol_k_col = cols_lower['vol_kospi']
vol_s_col = cols_lower['vol_samsung']

sigma_values = df_filtered[[vol_k_col, vol_s_col]].to_numpy()

# 제대로 필터링되었는지 확인하는 코드
print('Zero rate cols used:', zero_rate_cols)
print('zero_rate shape:', zero_rate.shape)
print('r-q columns:', [rq_k_col, rq_s_col], '-> r-q shape:', r_minus_q.shape)
print('vol columns:', [vol_k_col, vol_s_col], '-> vol shape:', sigma_values.shape)


In [ ]:

# ===============================================================
# 3) ELS pricing & Delta
# ===============================================================
class ELS2Star4838:
    def __init__(self,
                 T=3.0, gap=0.5,
                 step_down=(0.92, 0.90, 0.85, 0.85, 0.80),
                 redemp_pay=(1.0275, 1.0550, 1.0825, 1.1100, 1.1375),
                 maturity_pay=1.1650,
                 ki=0.60,
                 sigma=None,
                 rho=0.7477,
                 drift=None,            # << drift = (r−q) 넣는 곳
                 S0=(1.0, 1.0),
                 n_paths = 200000,
                 seed=42):

        self.T = T
        self.gap = gap
        self.obs = np.arange(gap, T+1e-12, gap)[:len(step_down)]
        self.step_down = np.array(step_down)
        self.redemp_pay = np.array(redemp_pay) * 10000
        self.maturity_pay = maturity_pay * 10000
        self.ki = ki

        self.sigma = np.array(sigma)
        self.rho = rho
        self.drift = np.array(drift)
        self.S0 = np.array(S0)

        self.n_paths = n_paths
        self.rng = np.random.default_rng(seed)

        corr = np.array([[1, rho], [rho, 1]])
        D = np.diag(self.sigma)
        cov = D @ corr @ D
        self.L = np.linalg.cholesky(cov)

    # -----------------------------
    # 시뮬레이션 (drift = r−q)
    # -----------------------------
    def _simulate(self, S0, T_remain, steps_per_year=252):

        dt = 1/steps_per_year
        n_steps = int(T_remain * steps_per_year)

        paths = np.zeros((self.n_paths, n_steps+1, 2))
        paths[:, 0] = S0

        drift = (self.drift - 0.5*self.sigma**2) * dt
        sdt = np.sqrt(dt)

        Z = self.rng.standard_normal((self.n_paths, n_steps, 2))
        dW = Z @ self.L.T

        for t in range(n_steps):
            incr = np.exp(drift + sdt * dW[:, t])
            paths[:, t+1] = paths[:, t] * incr

        return paths
    # -----------------------------
    # 가격 계산
    # -----------------------------
    def price(self, t0, S0_current, past_min, steps_per_year=252):
        """
        t0: 현재 시점 (년 단위)
        S0_current: (S1_t0, S2_t0)
        past_min: (min1, min2)  # t0까지의 최소값
        """

        # 남은 기간
        T_remain = self.T - t0
        if T_remain <= 0:
            raise ValueError("t0 must be earlier than maturity.")

        # 남은 관측일
        remain_obs = self.obs[self.obs > t0]
        obs_idx = ((remain_obs - t0) * steps_per_year).astype(int)

        # 중간 시점 기준으로 시뮬레이션
        paths = self._simulate(S0_current, T_remain, steps_per_year)
        n = len(paths)

        # 과거 최소와 이후 최소를 합쳐 낙인 여부 판정
        future_min = paths.min(axis=1).min(axis=1)
        ki_touch = np.minimum(future_min, np.min(past_min)) < self.ki

        payoff = np.zeros(n)
        redeemed = np.zeros(n, dtype=bool)

        # 남은 조기상환만 체크
        for i, idx in enumerate(obs_idx):
            cond = (~redeemed) & np.all(paths[:, idx] >= self.step_down[i + len(self.obs)-len(remain_obs)], axis=1)
            payoff[cond] = self.redemp_pay[i + len(self.obs)-len(remain_obs)] * discount_factor(remain_obs[i])
            redeemed |= cond

        not_red = ~redeemed
        if np.any(not_red):
            worst_T = paths[not_red, -1].min(axis=1)
            ki_hit = ki_touch[not_red]

            p = np.zeros_like(worst_T)
            p[(worst_T >= 0.70)] = self.maturity_pay
            p[(worst_T < 0.70) & (~ki_hit)] = self.maturity_pay
            p[(worst_T < 0.70) & (ki_hit)] = 10000 * worst_T[(worst_T < 0.70) & (ki_hit)]

            payoff[not_red] = p * discount_factor(T_remain)

        return payoff.mean(), payoff.std(ddof=1)/np.sqrt(n)
        
    def delta(self, t0, S0_current, past_min, h=0.01, 
            steps_per_year=252):
        """
        기초자산 변화에 대한 델타 계산 (중앙 차분)
        S0_current: (S1, S2)
        h: bump 크기 (예: 0.01 = 1%)
        """

        S1, S2 = S0_current

        # Seed 보존
        original_rng = self.rng
        self.rng = np.random.default_rng(12345)  # 공통 난수 사용

        # Δ1 계산
        V_up = self.price(t0, (S1*(1+h), S2), past_min, steps_per_year=steps_per_year)[0]
        V_dn = self.price(t0, (S1*(1-h), S2), past_min, steps_per_year=steps_per_year)[0]
        delta1 = (V_up - V_dn) / (2 * S1 * h)

        # Δ2 계산
        V_up2 = self.price(t0, (S1, S2*(1+h)), past_min, steps_per_year=steps_per_year)[0]
        V_dn2 = self.price(t0, (S1, S2*(1-h)), past_min, steps_per_year=steps_per_year)[0]
        delta2 = (V_up2 - V_dn2) / (2 * S2 * h)

        # rng 원상복구
        self.rng = original_rng

        return delta1, delta2

In [ ]:
# ===============================================================
# 4) Date별 pricing 및 delta 계산
# ===============================================================

model = ELS2Star4838(
    drift=r_minus_q,
    sigma=sigma_values
)

# Date기준 for문 돌리기 필요(아래 예시코드 참조해서 만들면 될듯?)

t0 = 1.0
S0_current = (0.92, 1.05) # 기초자산 가격이지만 중요할까?
past_min = (0.88, 0.95)

price = model.price(t0, S0_current, past_min)
delta1, delta2 = model.delta(t0, S0_current, past_min)

print(price)
print("Delta w.r.t S1 =", delta1)
print("Delta w.r.t S2 =", delta2)